### 1. What is the approximate depth of a decision tree trained (without restrictions) on a training set with one million instances? 

If a well-balanced decision tree contains m leaves, the depth is given by $log_{2}(m)$.  All of the decision trees in scikit-learn are binary trees, so assuming our tree is balanced at the end of training and with one leaf per training instance, if we have 1,000,000 instances, the depth is $log_{2}(10^{6})$ = 20, but in reality a little bit higher since the tree will not generally be completely balanced. 

### 2. Is a node's Gini impurity generally lower or greater than its parents'? Is it *generally* lower/greater, or *always* lower/greater?

The Gini impurity is *generally* lower than its parents'.  This is due to the CART cost function, which splits each node such that the weighted sum of the children's Gini impurities is minimized.  But it is possible for a node to have a higher Gini impurity than its parent, but only if this increase is compensated by a decrease in the other child's impurity.  

### 3. If a decsion tree is overfitting, is it a good idea to try decreasing `max_depth`? 

Yes.  The `max_depth` parameter controls the maximum depth of the decision tree.  We can tune it to implement a form of regularization and control the bias/variance tradeoff of the tree.  By decreasing `max_depth`, we will constrain the model, thus regularizing it.  

### 4. If a decision tree is underfitting the training set, is it a good idea to try scaling the input features? 

No.  We do not need to worry about scaling/centering/normalizing in decision trees.  If a decision tree is underfitting, it will be a waste of time to bother with any such data transformation, which is definitely not the case in other models where we would care. 

### 5. If it takes one hour to train a decision tree on a training set containing one million instances, roughly how much time will it take to train another decision tree on a training set containing ten million instances? 

The computational complexity of training a decision tree is O(n x m log(m)), so if the training set is 10x bigger, then the training time will be multiplied by K = (n x 10m x log(10m)) / (n x m x log(m)) = 10log(10m) / log(m).  So for m = 10^6, K is about 11.7, so 11.7 hours. 

### 6. If a training set contains 100,000 instances, will setting `presort=True` speed up training? 

Presorting a training set speed sup training only if the dataset is smaller than a few thousand instances.  If it has 100,000 instances, setting this will significantly slow down training. 

### 7. Train and fine-tune a decision tree for the moons dataset by following these steps: 

1. Use `make_moons(n_samples=10000, noise=0.4)` to generate a moons dataset

In [5]:
from sklearn.datasets import make_moons
X, y = make_moons(n_samples=10000, noise=0.4, random_state=1)

2. Use `train_test_split()` to split the dataset into a training and test set

In [6]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)

3. Use grid search with cross-validation to find good hyperparameter values for a `DecisionTreeClassifier`. 

In [7]:
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier
params = {'max_leaf_nodes': list(range(2, 100)), 'min_samples_split': [2, 3, 4]}
grid_search_cv = GridSearchCV(DecisionTreeClassifier(random_state=1), params, cv=3)
grid_search_cv.fit(X_train, y_train)

GridSearchCV(cv=3, estimator=DecisionTreeClassifier(random_state=1),
             param_grid={'max_leaf_nodes': [2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                            13, 14, 15, 16, 17, 18, 19, 20, 21,
                                            22, 23, 24, 25, 26, 27, 28, 29, 30,
                                            31, ...],
                         'min_samples_split': [2, 3, 4]})

4. Train it on the full training set using these hyperparameters and measure the model performance on the test set.  We should expect about 85-87% accuracy. 

In [8]:
grid_search_cv.best_estimator_

DecisionTreeClassifier(max_leaf_nodes=21, random_state=1)

By default, `GridSearchCV` trains the best model found on the whole training set (you can change this by setting `refit=False`), so we don't need to do it again. We can simply evaluate the model's accuracy:

In [9]:
from sklearn.metrics import accuracy_score

y_pred = grid_search_cv.predict(X_test)
accuracy_score(y_test, y_pred)

0.8435

### 8. Grow a forest by following these steps: 

1. Generate 1,000 subsets of the training set, each containing 100 instances selected randomly, following the last problem.

In [10]:
from sklearn.model_selection import ShuffleSplit

n_trees = 1000
n_instances = 100

mini_sets = []

rs = ShuffleSplit(n_splits=n_trees, test_size=len(X_train) - n_instances, random_state=1)
for mini_train_index, mini_test_index in rs.split(X_train):
    X_mini_train = X_train[mini_train_index]
    y_mini_train = y_train[mini_train_index]
    mini_sets.append((X_mini_train, y_mini_train))

2. Train one decision tree on each subset, using the best hyperparameter values found in the previous exercise.  Evaluate these 1,000 decision trees on the test set.  Since they were trained on smaller sets, these decision trees will likely perform worse than the first decision tree, achieving only about 80% accuracy.  

In [12]:
import numpy as np
from sklearn.base import clone
forest = [clone(grid_search_cv.best_estimator_) for _ in range(n_trees)]

accuracy_scores = []

for tree, (X_mini_train, y_mini_train) in zip(forest, mini_sets):
    tree.fit(X_mini_train, y_mini_train)
    
    y_pred = tree.predict(X_test)
    accuracy_scores.append(accuracy_score(y_test, y_pred))

np.mean(accuracy_scores)

0.7802165

3. For each test set instance, generate the predictions of the 10000 decision trees and keep only the most frequent prediction.  This approach gives us the *majority-vote predictions* over the test set. 

In [13]:
Y_pred = np.empty([n_trees, len(X_test)], dtype=np.uint8)

for tree_index, tree in enumerate(forest):
    Y_pred[tree_index] = tree.predict(X_test)
    
from scipy.stats import mode

y_pred_majority_votes, n_votes = mode(Y_pred, axis=0)

4. Evaluate these predictions on the test set: we should get a slightly higher accuracy than our first model.  

In [14]:
accuracy_score(y_test, y_pred_majority_votes.reshape([-1]))

0.8505